In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sbi
import os
import sys

def set_N_threads(N_threads=1):
    
    print('N_threads: %s'%(N_threads))
    
    os.environ["OMP_NUM_THREADS"] = str(N_threads)
    os.environ["OPENBLAS_NUM_THREADS"] = str(N_threads)
    os.environ["MKL_NUM_THREADS"] = str(N_threads)
    os.environ["VECLIB_MAXIMUM_THREADS"] = str(N_threads)
    os.environ["NUMEXPR_NUM_THREADS"] = str(N_threads)
    
    return N_threads

N_threads = set_N_threads()

In [ ]:
sys.path.append("/lscratch/dlopez/Projects/muchisimocks/scripts")
import sbi_tools

In [ ]:
biases_vec = np.loadtxt('/lscratch/mzennaro/Notebooks/muchisimocks/scripts/biases_vec.txt')

indicesLH = []
for ii, dir_name in enumerate(os.listdir('/lscratch/mzennaro/Notebooks/muchisimocks/scripts')):
    if dir_name.split('_')[0] == "biased":
        indicesLH.append(int(dir_name.split('_')[4]))
indicesLH = np.array(indicesLH)
indicesLH = np.unique(indicesLH)

fig, ax = plt.subplots(figsize=(8, 6))

Pk = []
for iLH in range(len(indicesLH)):
    for ib in range(len(biases_vec)):
        kk, tmp_pk = np.loadtxt('/lscratch/mzennaro/Notebooks/muchisimocks/scripts/biased_pk_m2m_num_%04d_bias_num_%d.txt'%(indicesLH[iLH], ib), unpack=True)
        Pk.append(tmp_pk)
        mask = tmp_pk > 0
        ax.loglog(kk[mask], tmp_pk[mask], label='bias %d'%ib)
        
Pk = np.array(Pk)
        
ax.set_xlabel(r'$k \,\, [h \,\, {\rm Mpc}^{-1}]$', fontsize=23)
ax.set_ylabel(r'$P(k) \,\, [h^{-3} \,\, {\rm Mpc}^3]$', fontsize=23)

mpl.pyplot.show()
print('finished')

In [ ]:
path_read_theta = "/dipc_storage/mpelle/Yin_data/Quijote"

theta = []
for iLH, id_LH in enumerate(indicesLH):
    theta.append(np.loadtxt(
        os.path.join(
            path_read_theta, "LH" + str(id_LH).zfill(4), "param_"+str(id_LH).zfill(4)+".txt")
    ))
theta = np.array(theta)
theta = np.repeat(theta, 10, axis=0)

In [ ]:
fig, ax = mpl.pyplot.subplots(1,1, figsize=(4,4))
ax.set_yscale('log')
ax.plot(Pk[0:9].T)
print(theta[0:10])

In [ ]:
train_val_split = int(theta.shape[0]*0.8)

theta_train = theta[:train_val_split]
theta_test = theta[train_val_split:]

Pk_train = Pk[:train_val_split]
Pk_test = Pk[train_val_split:]

In [ ]:
dict_bounds = {
    'omega_m'    :  [np.min(theta, axis=0)[0], np.max(theta, axis=0)[0]],
    'omega_b'    :  [np.min(theta, axis=0)[1], np.max(theta, axis=0)[1]],
    'hubble'     :  [np.min(theta, axis=0)[2], np.max(theta, axis=0)[2]],
    'ns'         :  [np.min(theta, axis=0)[3], np.max(theta, axis=0)[3]],
    'sigma8'     :  [np.min(theta, axis=0)[4], np.max(theta, axis=0)[4]],
}

In [ ]:
inference, posterior = sbi_tools.train_model(
    theta_train,
    Pk_train,
    prior= sbi_tools.get_prior(dict_bounds)
)

In [ ]:
# ------------------ posterior inference ------------------ #

# norm_xx_test = scaler.transform(xx_test)

inferred_theta_test = sbi_tools.sample_posteriors_theta_test(
    posterior,
    Pk_test,
    dict_bounds
)

# ------------------ rank stats ------------------ #

ranks = sbi_tools.compute_ranks(theta_test, inferred_theta_test)

In [ ]:
fig, ax = mpl.pyplot.subplots(1,1, figsize=(7,5))
fontsize = 24
fontsize1 = 18

tmp_Pk_plot = Pk_train
tmp_Pk_plot = tmp_Pk_plot[np.random.choice(tmp_Pk_plot.shape[0], tmp_Pk_plot.shape[0], replace=False)].T
ax.plot(np.log10(kk), np.log10(tmp_Pk_plot), c='royalblue', alpha=0.2, lw=0.5)

tmp_Pk_plot = Pk_test
tmp_Pk_plot = tmp_Pk_plot[np.random.choice(tmp_Pk_plot.shape[0], tmp_Pk_plot.shape[0], replace=False)].T
ax.plot(np.log10(kk), np.log10(tmp_Pk_plot), c='k', alpha=0.1, lw=0.5)
    
mpl.pyplot.tight_layout()
mpl.pyplot.show()

In [ ]:
# N_examples = 2

# custom_titles = [
#     r'$\Omega_\mathrm{m}$',
#     r'$\Omega_\mathrm{b}$',
#     r'$h$',
#     r'$n_\mathrm{s}$',
#     r'$\sigma_{8}$'
# ]

# colors = sbibe.plot_utils.get_N_colors(N_examples, mpl.colormaps['prism'])
# for ii_sample in range(N_examples):
#     fig, axs = sbibe.plot_utils.corner_plot(
#         theta_test[ii_sample],
#         inferred_theta_test[ii_sample],
#         custom_titles,
#         dict_bounds,
#         color_infer=colors[ii_sample]
#     )
#     mpl.pyplot.show()
    
# fig,save("popopo.png")

In [ ]:
# fig, axs = sbibe.plot_utils.plot_parameter_prediction_vs_truth(inferred_theta_test, theta_test, custom_titles)
# # plt.tight_layout()
# # plt.subplots_adjust(wspace=0.6)
# # plt.show()

In [ ]:
# fig, axs = sbibe.plot_utils.plot_rank_statistcis(ranks, inferred_theta_test.shape[1], custom_titles)
# # plt.tight_layout()
# # plt.subplots_adjust(wspace=0.05)
# # plt.show()